In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
from konlpy.tag import Kkma
kkma = Kkma()
#문단 -> 문장
para = "문자열 사용 방법 알려주세요. 문자열 초기화 방법 몰라. 문자열 출력은 어떻게 해?"
ex_sent = kkma.sentences(para)
len(ex_sent) #문장 2개
ex_sent

['문자열 사용 방법 알려 주세요.', '문자열 초기화 방법 몰라. 문자열 출력은 어떻게 해?']

In [3]:
# 문장 -> 단어(nouns)
from re import match
nouns = []
for sent in ex_sent:
    for noun in kkma.nouns(sent):
        if len(str(noun)) >= 2 and (match('[^방법]', noun)) :
            nouns.append(noun)
    
nouns

['문자열', '사용', '문자열', '초기화', '출력']

In [2]:
data = pd.read_csv('C:/Users/user/Desktop/recommend_dataset.csv', low_memory=False)
data[['genres']].head()
data[['keywords']].head()

,keywords
0,"{'문자열', '출력'}"
1,"{'문자열', '초기화'}"
2,"{'조건문', 'if'}"
3,"{'조건문', 'if', 'else'}"
4,"{'조건문', 'else'}"


In [3]:
import ast as ast
data['genres'] = data['genres'].apply(ast.literal_eval)
data[['genres']].head()

data['keywords'] = data['keywords'].apply(ast.literal_eval)
data[['keywords']].head()

,keywords
0,"{출력, 문자열}"
1,"{초기화, 문자열}"
2,"{조건문, if}"
3,"{조건문, if, else}"
4,"{조건문, else}"


In [5]:
data['genres'] = data['genres'].apply(lambda x:[d['name'] for d in x]).apply(lambda x:" ".join(x))
data['keywords'] = data['keywords'].apply(lambda x:[d for d in x]).apply(lambda x:" ".join(x))
data.head()

,ques_id,ques_content,language,category,sub_category,keywords_origin,keywords,genres
0,1,문자열 출력 방법,C,문자열,출력,"문자열, 출력",출력 문자열,Animation Comedy Family
1,2,문자열 초기화,C,문자열,초기화,"문자열, 초기화",초기화 문자열,Adventure Fantasy Family
2,3,if 문,C,조건문,NaN,조건문,조건문 if,Romance Comedy
3,4,if else문,C,조건문,NaN,조건문,조건문 if else,Comedy Drama Romance
4,5,else 문,C,조건문,NaN,조건문,조건문 else,Comedy


In [6]:
#keywords에 대해 tf-idf 수행

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['keywords'])
print(tfidf_matrix.shape)

(17, 11)


In [7]:
#코사인 유사도 구하기
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

indices = pd.Series(data.index, index=data['ques_content']).drop_duplicates()
print(indices.head())

idx = indices['if 문']
print(idx)

[[1.         0.34137367 0.         0.         0.         0.
  0.39080903 0.         0.         0.27571797 0.         1.
  1.         1.         1.         1.         1.        ]
 [0.34137367 1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.34137367
  0.34137367 0.34137367 0.34137367 0.34137367 0.34137367]
 [0.         0.         1.         0.79064598 0.40030986 0.36745114
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.79064598 1.         0.79064598 0.29052376
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.40030986 0.79064598 1.         0.36745114
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.36745114 0.29052376 0.36745114 1.
  0.         0.   

In [8]:
#사용자의 질문에 대해 코사인 유사도를 이용하여
#가장 유사도가 비슷한 질문을 찾아내는 함수
def get_recommendations(question):
    #선택한 질문의 인덱스 구하기
    idx = indices[question]
    
    #모든 질문에 대해 해당 질문과의 유사도 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    #유사도에 따라 질문들 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #가장 유사한 5개 질문 받아오기
    sim_scores = sim_scores[1:5]
    
    #가장 유사한 5개의 질문의 인덱스 받아오기
    ques_indices = [i[0] for i in sim_scores]
    
    #가장 유사한 5개의 질문 리턴
    return data['ques_content'].iloc[ques_indices]

In [9]:
# '반복문 종료' 과 비슷한 질문들 찾기
get_recommendations('반복문 종료')

10      for문 종료
6       반복문 출력 
8     while문 종료
9       for문 출력
Name: ques_content, dtype: object